In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [4]:
# run optimization with constraints
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

In [5]:
result.params

[0.8234836645460724,
 0.26711162818357315,
 -0.5518536233851673,
 -0.06099135116078457,
 array([0.58284494, 0.72495343])]

In [6]:
result.gamma()

-0.5518536233851673

In [7]:
result.residual_mean()

-0.014427880456180125

In [8]:
result.residual_std()

0.7976726564809411

In [9]:
result.asymp_var()

array([[ 0.27532024, -0.11183192,  0.17212879, -0.09062966, -0.23770971,
        -0.12794589],
       [-0.11183192,  0.68277726,  0.16952129, -0.48552553,  0.01427696,
        -0.01357915],
       [ 0.17212879,  0.16952129,  0.46630032, -0.35976425, -0.17389169,
        -0.09755707],
       [-0.09062966, -0.48552553, -0.35976425,  0.67135597,  0.16575271,
         0.10820594],
       [-0.23770971,  0.01427696, -0.17389169,  0.16575271,  0.30277952,
         0.17306741],
       [-0.12794589, -0.01357915, -0.09755707,  0.10820594,  0.17306741,
         0.21644569]])

In [10]:
result.print_table()

Residuals:
     Min         1Q     Median        3Q      Max
--------  ---------  ---------  --------  -------
-1.64196  -0.442753  0.0138896  0.431488  1.27584

Estimated Parameters:
   Sigma    Lambda      Gamma         Rho  Beta
--------  --------  ---------  ----------  -----------------------
0.823484  0.267112  -0.551854  -0.0609914  [0.58284494 0.72495343]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
 0.261146  0.0697027    0.25164  -0.582489

Estimated c0 (display first 10 rows): 
    c0_0      c0_1      c0_2       c0_3
--------  --------  --------  ---------
0.775571  0.721358  0.599133  -0.234643

Estimated asympotitic variance: 
              Gamma         Rho      Beta_0      Beta_1     Lambda     Sigma^2
-------  ----------  ----------  ----------  ----------  ---------  ----------
Gamma     0.27532    -0.111832    0.172129   -0.0906297  -0.23771   -0.127946
Rho      -0.111832    0.682777  

In [12]:
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, constrain= False)

In [13]:
result.print_table()

Residuals:
     Min         1Q     Median        3Q      Max
--------  ---------  ---------  --------  -------
-1.64196  -0.442753  0.0138887  0.431489  1.27584

Estimated Parameters:
   Sigma    Lambda      Gamma        Rho  Beta
--------  --------  ---------  ---------  -----------------------
0.823484  0.267113  -0.551855  -0.060991  [0.58284467 0.72495197]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
 0.261146  0.0697042    0.25164   -0.58249

Estimated c0 (display first 10 rows): 
    c0_0      c0_1      c0_2       c0_3
--------  --------  --------  ---------
0.775571  0.721357  0.599133  -0.234645

Estimated asympotitic variance: 
              Gamma         Rho      Beta_0      Beta_1      Lambda     Sigma^2
-------  ----------  ----------  ----------  ----------  ----------  ----------
Gamma     0.275321   -0.111833    0.172129   -0.0906291  -0.23771    -0.127946
Rho      -0.111833    0.682778  

### 2. Test k=1

In [14]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 20
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [15]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [16]:
result.print_table()

Residuals:
     Min         1Q    Median       3Q      Max
--------  ---------  --------  -------  -------
-2.69784  -0.526343  0.271297  1.19655  4.77589

Estimated Parameters:
   Sigma    Lambda       Gamma       Rho      Beta
--------  --------  ----------  --------  --------
0.957797  0.459022  -0.0175539  0.262422  0.905975

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3    alpha_4    alpha_5    alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 -0.20673    1.32436  -0.865887   0.696435   0.153478  -0.192189  -0.318961   0.268251   0.177229   -1.03599

Estimated c0 (display first 10 rows): 
     c0_0       c0_1       c0_2       c0_3      c0_4       c0_5       c0_6      c0_7       c0_8        c0_9
---------  ---------  ---------  ---------  --------  ---------  ---------  --------  ---------  ----------
-0.811652  -0.530314  -0.160762  -0.438911 

In [17]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [18]:
result.print_table()

Residuals:
     Min         1Q    Median       3Q      Max
--------  ---------  --------  -------  -------
-2.69784  -0.526343  0.271297  1.19655  4.77589

Estimated Parameters:
   Sigma    Lambda       Gamma       Rho      Beta
--------  --------  ----------  --------  --------
0.957797  0.459022  -0.0175539  0.262422  0.905975

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3    alpha_4    alpha_5    alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 -0.20673    1.32436  -0.865887   0.696435   0.153478  -0.192189  -0.318961   0.268251   0.177229   -1.03599

Estimated c0 (display first 10 rows): 
     c0_0       c0_1       c0_2       c0_3      c0_4       c0_5       c0_6      c0_7       c0_8        c0_9
---------  ---------  ---------  ---------  --------  ---------  ---------  --------  ---------  ----------
-0.811652  -0.530314  -0.160762  -0.438911 